## <center>Analiza poziomu PM2.5 w afrykańskich miastach</center>
### Zespół:
<ol>
    <li style='font-size: 20px'>Hubert Kłosowski 242424</li>
    <li style='font-size: 20px'>Krzysztof Kolanek 242425</li>
    <li style='font-size: 20px'>Kamil Małecki 242464</li>
</ol>

### Potrzebne importy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn, optim
from sklearn.impute import KNNImputer
from sympy import divisors
from scipy.stats import zscore
from sklearn.feature_selection import SelectKBest, RFECV, RFE, mutual_info_regression, f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import root_mean_squared_error
import optuna

### Wczytanie danych

In [ ]:
data = pd.read_csv('data\\Train.csv')
test = pd.read_csv('data\\Test.csv')

data.info()

In [ ]:
data.head()

### Rozbicie daty na składowe

In [ ]:
def change_date(dataframe):
    dataframe['date'] = pd.to_datetime(dataframe['date'])
    dataframe['day'] = dataframe['date'].dt.dayofweek.astype(np.int64)
    dataframe['month'] = dataframe['month'].astype(np.int64)
    return dataframe


data, test = change_date(data), change_date(test)

### Wykres przedstawiający jakość powietrza w krajach afrykańskich

In [ ]:
sns.lineplot(data=data, x='date', y='pm2_5', hue='country')
plt.title('Jakość powietrza z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 w zarejestrowanych godzinach

In [ ]:
sns.barplot(data=data, x='hour', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych godzinach z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od dnia tygodnia

In [ ]:
sns.barplot(data=data, x='day', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od miesiąca

In [ ]:
sns.barplot(data=data, x='month', y='pm2_5', hue='country')
plt.title('Jakość powietrza w każdym dniu tygodnia z podziałem na kraje')

### Korelacje poszczególnych grup kolumn

In [ ]:
def correlation():
    for i, column in enumerate(starts_with):
        selected_columns = [col for col in data.columns if col.startswith(column) or col == 'pm2_5']
        if len(selected_columns) > 1:
            fig, ax = plt.subplots(figsize=(10, 10))
            sns.heatmap(data[selected_columns].corr(), annot=True, fmt='.2f', cmap='viridis', ax=ax)
            plt.tight_layout()
            plt.show()
        
def drop_high_correlated_columns(dataframe):
    matrix = dataframe.corr(numeric_only=True)
    upper = matrix.where(np.triu(np.ones(matrix.shape), k=1).astype(np.bool_))
    to_drop = [column for column in upper.columns if any(upper[column] >= 0.9)]
    return dataframe.drop(to_drop, axis=1)


final_ids = test['id']
starts_with = data.columns.str.split('_', expand=True).levels[0].to_frame()
starts_with.drop(['month', 'day', 'hour', 'pm2'], inplace=True)
starts_with = starts_with[0].tolist()
data, test = drop_high_correlated_columns(data), drop_high_correlated_columns(test)
data.drop(columns=['id', 'city', 'country', 'site_id', 'date'], inplace=True)
test.drop(columns=['id', 'city', 'country', 'site_id', 'date'], inplace=True)

correlation()

## <center>Czyszczenie danych</center>

### 1. Uzupełnienie wartości brakujących

In [ ]:
def fill_based_on(dataframe, date_unit='day'):
    date_range = dataframe[date_unit].unique()
    for date in date_range:
        for i, column in enumerate(starts_with):
            similar_columns = [el for el in dataframe.columns if el.startswith(column)]
            df = dataframe.loc[dataframe[date_unit] == date, similar_columns]
            if not df.empty:
                dataframe.loc[dataframe[date_unit] == date, similar_columns] = imputers[i].fit_transform(df)
    return dataframe

def prepare_dataframe(dataframe):  # usuwamy kolumny o dużej liczbie wartości NaN
    to_drop = []
    for i, el in enumerate(dataframe.columns):
        if dataframe[el].isna().sum() / len(dataframe) >= 0.9:
            to_drop.append(el)
    dataframe.drop(to_drop, axis=1, inplace=True)
    return dataframe


imputers = [KNNImputer(n_neighbors=15, weights='distance') for _ in range(len(starts_with))]
data, test = prepare_dataframe(data), prepare_dataframe(test)
data, test = fill_based_on(data), fill_based_on(test)
vertical_columns = [col for col in data.columns if 'number_density' in col]
ver = dict(zip(vertical_columns, [(abs(data[column].quantile(0.1)) * -1.5 / data[column].std(), abs(data[column].quantile(0.9)) * 1.5 / data[column].std()) for column in vertical_columns]))

### Wykresy pudełkowe wskazujące wartości odstające

In [ ]:
def plot_boxplots():
    for i, column_group in enumerate(starts_with):
        similar_columns = [col for col in data.columns if col.startswith(column_group)]
        if len(similar_columns) > 1:
            divs = divisors(len(similar_columns))
            if len(divs) % 2 == 0:
                rows, cols = divs[(len(divs) // 2) - 1], divs[len(divs) // 2]
            else:
                rows, cols = divs[len(divs) // 2], divs[len(divs) // 2]
            fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(40, 30), squeeze=False)
            fig.suptitle(column_group, fontsize=25)
            for j, column in enumerate(similar_columns):
                x_cord, y_cord = divmod(j, cols)
                data[column].plot(kind='box', ax=ax[x_cord, y_cord], fontsize=15)
                if column in ver.keys():
                    ax[x_cord, y_cord].axhline(y=ver.get(column)[0] * data[column].std(), color='green', label='down-limit')
                    ax[x_cord, y_cord].axhline(y=ver.get(column)[1] * data[column].std(), color='red', label='up-limit')
                ax[x_cord, y_cord].legend(loc='best')
            plt.show()


plot_boxplots()

### 2. Usunięcie wartości odstających

In [ ]:
def del_outliers(dataframe, ignore_down_range=True):
    for column, zscore_range in ver.items():
        vec, indexes = zscore(dataframe[column]), []
        for j in range(len(vec)):
            if ignore_down_range:
                if vec[j] >= zscore_range[1]:
                    indexes.append(j)
            else:
                if zscore_range[0] <= vec[j] >= zscore_range[1]:
                    indexes.append(j)
        dataframe.drop(index=indexes, inplace=True)
        dataframe.reset_index(drop=True, inplace=True)
    return dataframe


data = del_outliers(data)

data.info()

In [ ]:
data.head()

## <center>Selekcja cech</center>

In [ ]:
def plot_feature_importance(sc, num_of_features):
    if isinstance(sc, RFECV) or isinstance(sc, RFE):
        scores = dict(zip(sc.feature_names_in_, sc.ranking_))
    else:
        scores = dict(zip(sc.feature_names_in_, sc.scores_))
    scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:num_of_features]
    scores_df = pd.DataFrame(scores, columns=['Feature', 'Score'])
    
    scores_df.plot(kind='bar', x='Feature', y='Score', figsize=(10, 6), rot=90, title='Oceny wybranych cech')
    plt.xlabel('Cecha')
    plt.ylabel('Ocena')


X, y = data.drop(['pm2_5'], axis=1), data['pm2_5']
# selector = RFE(
#     estimator=RandomForestRegressor(
#         n_estimators=700, 
#         max_depth=7, 
#         random_state=4, 
#         n_jobs=-1, 
#         oob_score=True,
#         warm_start=True
#     ),
#     n_features_to_select=k,
# )
k = 25
selector = RFECV(
    estimator=RandomForestRegressor(
        n_estimators=400, 
        max_depth=10, 
        random_state=4, 
        n_jobs=-1, 
        oob_score=True,
        warm_start=True,
    ),
    min_features_to_select=k, 
    cv=10, 
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)
selector.fit(X, y)
X, test = selector.transform(X), selector.transform(test)

plot_feature_importance(selector, k)

## <center>Transformacja danych</center>

### 1. Wybór sposobu preprocessingu danych

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X, y)
test = scaler.transform(test)

### 2. Podział na zbiór testowy i treningowy

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

## <center>Część obliczeniowa</center>

### Otrzymanie najlepszych parametrów

In [ ]:
def give_the_best(clf, param_grid):
    gs = GridSearchCV(clf, param_grid, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5)
    gs.fit(X_train, y_train)
    return gs.best_estimator_

def save_to_csv(y_pred, save_as):
    final_df = pd.concat([final_ids, pd.DataFrame.from_dict({'pm2_5': y_pred})], axis=1)
    final_df.to_csv(f'result\\{save_as}', index=False)

### <center>Regresja przy użyciu MLP</center>

In [ ]:
# params = {
#     'hidden_layer_sizes': [(99, 141, 75)],
#     'activation': ['relu'],
#     'solver': ['adam'],
#     'max_iter': [1000],
#     'alpha': np.linspace(0.0001, 0.001, 10),
#     'batch_size': [64, 128, 256],
#     'learning_rate_init': np.linspace(0.001, 0.01, 10),
#     'warm_start': [True],
#     'early_stopping': [True],
#     'validation_fraction': [0.1]
# }
# 
# mlp = give_the_best(MLPRegressor())
# save_to_csv(mlp.predict(test), 'mlp.csv')
# print('Parametry MLP: ', mlp.get_params())
# print('RMSE: ', root_mean_squared_error(y_test, mlp.predict(X_test)))

### <center>PyTorch</center>

### 1. Wybór karty graficznej do nauki modelu

In [ ]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

X_train_tensor = torch.tensor(X_train, device=device, dtype=torch.float)
X_test_tensor = torch.tensor(X_test, device=device, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.to_numpy(), device=device, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.to_numpy(), device=device, dtype=torch.float)
test_tensor = torch.tensor(test, device=device, dtype=torch.float)

### 1.1 Optuna

In [ ]:
def rmse_loss(y_true, y_pred):
    return torch.sqrt(torch.mean((y_true - y_pred) ** 2))

def define_model(trial):
    n_layers = trial.suggest_int('n_layers', 2, 5)
    layers = []
    
    in_features = X_train_tensor.shape[1]
    for i in range(n_layers):
        out_features = trial.suggest_int(f'layer{i}', 25, 200)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float(f'dropout{i}', 0, 0.4)
        layers.append(nn.Dropout(p))
        in_features = out_features
    layers.append(nn.Linear(in_features, 1))
    return nn.Sequential(*layers)

def objective(trial):
    model = define_model(trial).to(device=device)
    lr = trial.suggest_float('lr', 1e-3, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-4, log=True)
    betas = (trial.suggest_float('beta1', 0.8, 0.9, log=True), trial.suggest_float('beta2', 0.997, 0.999, log=True))
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay, betas=betas, fused=True)
    
    batch_size = trial.suggest_int('batch_size', 16, 416, step=25)
    criterion = rmse_loss
    epochs = trial.suggest_int('epochs', 100, 300, step=50)
    
    final_train_tensor = torch.concat((X_train_tensor, y_train_tensor.unsqueeze(dim=1)), dim=1)
    
    for epoch in range(epochs):
        dataset = DataLoader(final_train_tensor, batch_size=batch_size, shuffle=True)
        epoch_loss = 0
        for batch_idx, batch in enumerate(dataset):
            inputs, targets = batch[:, :-1], batch[:, -1]
            batch_pred = model(inputs)
            optimizer.zero_grad()
            loss = criterion(targets, batch_pred.squeeze())
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()
            
        trial.report(epoch_loss / len(dataset), epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    with torch.no_grad():
        y_pred = model(X_test_tensor).squeeze()
        
    return root_mean_squared_error(y_test, y_pred.numpy(force=True))

study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(),
    study_name='AirQuality',
)
study.optimize(objective, n_trials=200)

### Najlepsze parametry sieci neuronowej

In [ ]:
study.best_params

### Wykres przedstawiający każdy <i>trial</i> w procesie nauki

In [ ]:
optuna.visualization.plot_optimization_history(study)

### Wizualizacja przekroju parametrów

In [ ]:
optuna.visualization.plot_slice(study, params=['lr', 'n_layers', 'weight_decay', 'beta1', 'beta2', 'epochs', 'batch_size'])

### Zależność parametrów w postaci wykresu konturowego

In [ ]:
optuna.visualization.plot_contour(study, params=['lr', 'n_layers', 'beta1', 'beta2', 'epochs', 'batch_size'])

### Wykres przedstawiający wielowymiarowe zależności parametrów

In [ ]:
optuna.visualization.plot_parallel_coordinate(study, params=['lr', 'n_layers', 'beta1', 'beta2', 'epochs', 'batch_size'])

### Historia optymalizacji parametrów na osi czasu

In [ ]:
optuna.visualization.plot_timeline(study)

## <center>Do wysłania</center>

In [ ]:
best_model = define_model(study.best_trial)
best_model.to(device=device)

best_model.eval()

with torch.no_grad():
    save_to_csv(best_model(test_tensor).squeeze().numpy(force=True), 'nn.csv')

### Dodatkowe informacje
<ol>
    <li>The 15km SO2 band is ingested only when solar_zenith_angle < 70.</li>
    <li>Because of noise on the data, negative vertical column values are often observed in particular over clean regions or for low SO2 emissions. It is recommended not to filter these values except for outliers, i.e. for vertical columns lower than -0.001 mol/m^2.</li>
</ol>